In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools  # for cycling through colors if necessary
import glob
import re
import os

from scipy.stats import shapiro
from scipy.stats import levene

In [7]:
num_models = 4 #CHANGE
device = 'gpu'
path = f"D:/GAISSA/energy-repo/last_repo/june_{device}/"  # Adjust this path to your CSV files location #CHANGE


In [8]:
configs =['torch_pythia1-4b', 'torch_tinyllama',
       'torch_codeparrot-small', 'torch_pythia-410m', 'onnx_pythia1-4b',
       'onnx_tinyllama', 'onnx_codeparrot-small', 'onnx_pythia-410m',
       'torchscript_pythia1-4b', 'torchscript_tinyllama',
       'torchscript_codeparrot-small', 'torchscript_pythia-410m']

In [9]:
save_dir = path+"tests/"


try:
    os.mkdir(save_dir)
except Exception as e:
    print("could not create directory "+ save_dir )
    print(e)


could not create directory D:/GAISSA/energy-repo/last_repo/june_gpu/tests/
[WinError 183] Cannot create a file when that file already exists: 'D:/GAISSA/energy-repo/last_repo/june_gpu/tests/'


In [10]:
def remove_condition_rows(condition_to_remove,df):
    # example: condition = merged_df['label']  == 'idle_idle'
    df1 = df.copy()
    return df1[~condition_to_remove]

In [11]:
def save_latex_table(df,save_dir):
    df = df.copy()
    df.columns = [col.replace('_', ' ') for col in df.columns]
    
    latex_table = df.to_latex(index=False)

    # Define the filename
    filename = save_dir
    
    # Open the file in write mode
    with open(filename, 'w') as file:
        file.write(latex_table)
        
    # Print LaTeX table
    print(latex_table)

# Energibridge metrics

In [12]:
# Path where the CSV files are stored
all_files_energi = glob.glob(path + "/results_*/processed/energi_processed.csv")
all_files_energi

['D:/GAISSA/energy-repo/last_repo/june_gpu\\results_1\\processed\\energi_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_10\\processed\\energi_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_2\\processed\\energi_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_3\\processed\\energi_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_4\\processed\\energi_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_5\\processed\\energi_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_6\\processed\\energi_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_7\\processed\\energi_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_8\\processed\\energi_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_9\\processed\\energi_processed.csv']

In [13]:
result_numbers = []

li = []
for my_file in all_files_energi:
    # The regex pattern to find "results_" followed by one or more digits
    df = pd.read_csv(my_file, index_col=None, header=0)
    
    match = re.search(r'results_(\d+)', my_file)
    if match:
        # Append the found number to the list
        result_numbers.append(int(match.group(1)))

    df['experiment'] = result_numbers[-1]
    li.append(df)

print(result_numbers)
# Combine all data into one DataFrame
energi_df = pd.concat(li, axis=0, ignore_index=True)
energi_df

[1, 10, 2, 3, 4, 5, 6, 7, 8, 9]


,time,runtime,model,CPU_ENERGY (J),TOTAL_MEMORY,TOTAL_SWAP,USED_MEMORY,USED_SWAP,AVG_CPU_USAGE_SAMP,label,avg_TOTAL_MEMORY,avg_TOTAL_SWAP,avg_USED_MEMORY,avg_USED_SWAP,avg_cpu_usage_config,energy,avg_used_memory_pct_config,experiment
0,2024-06-08 21:42:00.400,idle,idle,516616.142639,66575933440,2147479552,2241388544,0,5.237742,idle_idle,6.657593e+10,2.147480e+09,2.794639e+09,0.0,0.143838,5356.125122,4.197672,1
1,2024-06-08 21:42:00.400,idle,idle,516616.207123,66575933440,2147479552,2246549504,0,5.237742,idle_idle,6.657593e+10,2.147480e+09,2.794639e+09,0.0,0.143838,5356.125122,4.197672,1
2,2024-06-08 21:42:00.600,idle,idle,516627.494354,66575933440,2147479552,2353451008,0,4.872685,idle_idle,6.657593e+10,2.147480e+09,2.794639e+09,0.0,0.143838,5356.125122,4.197672,1
3,2024-06-08 21:42:00.800,idle,idle,516639.257111,66575933440,2147479552,2402861056,0,4.872685,idle_idle,6.657593e+10,2.147480e+09,2.794639e+09,0.0,0.143838,5356.125122,4.197672,1
4,2024-06-08 21:42:01.000,idle,idle,516651.727112,66575933440,2147479552,2452594688,0,6.209758,idle_idle,6.657593e+10,2.147480e+09,2.794639e+09,0.0,0.143838,5356.125122,4.197672,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193376,2024-06-11 19:27:32.772,torchscript,phi2,458083.008850,66575949824,2147479552,12355092480,0,0.000000,torchscript_phi2,6.657595e+10,2.147480e+09,1.222203e+10,0.0,0.658319,2812.548523,18.358024,9
193377,2024-06-11 19:27:32.972,torchscript,phi2,458088.296036,66575949824,2147479552,12354756608,0,0.617560,torchscript_phi2,6.657595e+10,2.147480e+09,1.222203e+10,0.0,0.658319,2812.548523,18.358024,9
193378,2024-06-11 19:27:33.172,torchscript,phi2,458092.385895,66575949824,2147479552,12354756608,0,0.000000,torchscript_phi2,6.657595e+10,2.147480e+09,1.222203e+10,0.0,0.658319,2812.548523,18.358024,9
193379,2024-06-11 19:27:33.373,torchscript,phi2,458096.612167,66575949824,2147479552,12354756608,0,0.305060,torchscript_phi2,6.657595e+10,2.147480e+09,1.222203e+10,0.0,0.658319,2812.548523,18.358024,9


In [14]:
condition = energi_df['label'].str.endswith('phi2')

energi_df = remove_condition_rows( condition, energi_df)
energi_df

,time,runtime,model,CPU_ENERGY (J),TOTAL_MEMORY,TOTAL_SWAP,USED_MEMORY,USED_SWAP,AVG_CPU_USAGE_SAMP,label,avg_TOTAL_MEMORY,avg_TOTAL_SWAP,avg_USED_MEMORY,avg_USED_SWAP,avg_cpu_usage_config,energy,avg_used_memory_pct_config,experiment
0,2024-06-08 21:42:00.400,idle,idle,516616.142639,66575933440,2147479552,2241388544,0,5.237742,idle_idle,6.657593e+10,2.147480e+09,2.794639e+09,0.0,0.143838,5356.125122,4.197672,1
1,2024-06-08 21:42:00.400,idle,idle,516616.207123,66575933440,2147479552,2246549504,0,5.237742,idle_idle,6.657593e+10,2.147480e+09,2.794639e+09,0.0,0.143838,5356.125122,4.197672,1
2,2024-06-08 21:42:00.600,idle,idle,516627.494354,66575933440,2147479552,2353451008,0,4.872685,idle_idle,6.657593e+10,2.147480e+09,2.794639e+09,0.0,0.143838,5356.125122,4.197672,1
3,2024-06-08 21:42:00.800,idle,idle,516639.257111,66575933440,2147479552,2402861056,0,4.872685,idle_idle,6.657593e+10,2.147480e+09,2.794639e+09,0.0,0.143838,5356.125122,4.197672,1
4,2024-06-08 21:42:01.000,idle,idle,516651.727112,66575933440,2147479552,2452594688,0,6.209758,idle_idle,6.657593e+10,2.147480e+09,2.794639e+09,0.0,0.143838,5356.125122,4.197672,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192830,2024-06-11 19:25:43.190,torchscript,pythia-410m,455249.594849,66575949824,2147479552,9003851776,0,0.000000,torchscript_pythia-410m,6.657595e+10,2.147480e+09,9.000287e+09,0.0,0.278354,2176.047501,13.518826,9
192831,2024-06-11 19:25:43.390,torchscript,pythia-410m,455253.705002,66575949824,2147479552,9003851776,0,0.156250,torchscript_pythia-410m,6.657595e+10,2.147480e+09,9.000287e+09,0.0,0.278354,2176.047501,13.518826,9
192832,2024-06-11 19:25:43.591,torchscript,pythia-410m,455258.060974,66575949824,2147479552,9003851776,0,0.000000,torchscript_pythia-410m,6.657595e+10,2.147480e+09,9.000287e+09,0.0,0.278354,2176.047501,13.518826,9
192833,2024-06-11 19:25:43.791,torchscript,pythia-410m,455262.225143,66575949824,2147479552,9003851776,0,0.148810,torchscript_pythia-410m,6.657595e+10,2.147480e+09,9.000287e+09,0.0,0.278354,2176.047501,13.518826,9


## Normality test

In [15]:
cpu_energy_aggregated = energi_df.groupby(['label', 'experiment'])['energy'].mean().reset_index()
cpu_energy_aggregated

,label,experiment,energy
0,idle_idle,1,5356.125122
1,idle_idle,2,5350.155334
2,idle_idle,3,5174.852402
3,idle_idle,4,5180.039169
4,idle_idle,5,5366.871078
...,...,...,...
125,torchscript_tinyllama,6,2479.216080
126,torchscript_tinyllama,7,2528.263107
127,torchscript_tinyllama,8,2645.645905
128,torchscript_tinyllama,9,2569.570862


In [16]:
# Initialize an empty DataFrame to collect test results
results_df = pd.DataFrame(columns=['config', 'W-statistic', 'p-value'])

# Iterate over each configuration
for c in configs:
    # Filter data for current configuration
    subset = cpu_energy_aggregated[cpu_energy_aggregated['label'].str.startswith(c, na=False)]
    
    # Perform the Shapiro-Wilk test
    w_statistic, p_value = shapiro(subset['energy'])
    
    # Prepare a dictionary to add as a new row
    new_row = pd.DataFrame({'config': [c], 'W-statistic': [w_statistic], 'p-value': [p_value]})
    
    # Concatenate the new row to the results DataFrame
    results_df = pd.concat([results_df, new_row], ignore_index=True)

# Display the results DataFrame
print(results_df)



                          config  W-statistic   p-value
0               torch_pythia1-4b     0.944940  0.609167
1                torch_tinyllama     0.955115  0.729061
2         torch_codeparrot-small     0.981815  0.974134
3              torch_pythia-410m     0.945189  0.612056
4                onnx_pythia1-4b     0.501224  0.000004
5                 onnx_tinyllama     0.592261  0.000045
6          onnx_codeparrot-small     0.516940  0.000006
7               onnx_pythia-410m     0.754903  0.004095
8         torchscript_pythia1-4b     0.482679  0.000002
9          torchscript_tinyllama     0.951512  0.686412
10  torchscript_codeparrot-small     0.936205  0.511606
11       torchscript_pythia-410m     0.870550  0.101451


In [17]:
save_latex_table(results_df,save_dir + "cpu_energy_shapiro")

\begin{tabular}{lrr}
\toprule
config & W-statistic & p-value \\
\midrule
torch_pythia1-4b & 0.944940 & 0.609167 \\
torch_tinyllama & 0.955115 & 0.729061 \\
torch_codeparrot-small & 0.981815 & 0.974134 \\
torch_pythia-410m & 0.945189 & 0.612056 \\
onnx_pythia1-4b & 0.501224 & 0.000004 \\
onnx_tinyllama & 0.592261 & 0.000045 \\
onnx_codeparrot-small & 0.516940 & 0.000006 \\
onnx_pythia-410m & 0.754903 & 0.004095 \\
torchscript_pythia1-4b & 0.482679 & 0.000002 \\
torchscript_tinyllama & 0.951512 & 0.686412 \\
torchscript_codeparrot-small & 0.936205 & 0.511606 \\
torchscript_pythia-410m & 0.870550 & 0.101451 \\
\bottomrule
\end{tabular}



In [18]:
sets_levene = []
for c in configs:
    sets_levene.append(cpu_energy_aggregated[cpu_energy_aggregated['label'].str.startswith(c, na=False)]['energy'])

In [19]:
sets_levene[0]

70    9975.393097
71    9796.836838
72    9859.026871
73    9851.199417
74    9742.763779
75    9740.837982
76    9922.174866
77    9822.518509
78    9843.730118
79    9772.624771
Name: energy, dtype: float64

In [20]:
results_df = pd.DataFrame(columns=['W-statistic', 'p-value'])
w_statistic, p_value = levene(*sets_levene)
    
# Prepare a dictionary to add as a new row
new_row = pd.DataFrame({'W-statistic': [w_statistic], 'p-value': [p_value]})

# Concatenate the new row to the results DataFrame
results_df = pd.concat([results_df, new_row], ignore_index=True)

save_latex_table(results_df,save_dir + 'cpu_energy_levene')
# Display the results DataFrame
print(results_df)

\begin{tabular}{rr}
\toprule
W-statistic & p-value \\
\midrule
1.721887 & 0.077897 \\
\bottomrule
\end{tabular}

   W-statistic   p-value
0     1.721887  0.077897


In [21]:
print("")

# Nvidia-smi metrics

In [22]:
# Path where the CSV files are stored
path = "D:/GAISSA/energy-repo/last_repo/june_gpu/"  # Adjust this path to your CSV files location
all_files_nvidia = glob.glob(path + "/results_*/processed/nvidia*_processed.csv")
all_files_nvidia

['D:/GAISSA/energy-repo/last_repo/june_gpu\\results_1\\processed\\nvidia_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_10\\processed\\nvidia_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_2\\processed\\nvidia_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_3\\processed\\nvidia_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_4\\processed\\nvidia_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_5\\processed\\nvidia_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_6\\processed\\nvidia_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_7\\processed\\nvidia_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_8\\processed\\nvidia_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_9\\processed\\nvidia_processed.csv']

In [23]:
result_numbers = []

li = []
for my_file in all_files_nvidia:
    # The regex pattern to find "results_" followed by one or more digits
    df = pd.read_csv(my_file, index_col=None, header=0)
    
    match = re.search(r'results_(\d+)', my_file)
    if match:
        # Append the found number to the list
        result_numbers.append(int(match.group(1)))

    df['experiment'] = result_numbers[-1]
    li.append(df)

print(result_numbers)
# Combine all data into one DataFrame
nvidia_df = pd.concat(li, axis=0, ignore_index=True)
nvidia_df

[1, 10, 2, 3, 4, 5, 6, 7, 8, 9]


,time,runtime,model,utilization.gpu [%],utilization.memory [%],memory.total [MiB],memory.used [MiB],power.draw [W],power.max_limit [W],temperature.gpu,...,used_memory_pct_mib,avg_utilization_gpu_config,avg_utilization_memory_config,avg_memory_used_config,avg_power_draw_config,avg_temperature_gpu_config,avg_used_memory_pct_mib,config_duration,energy,experiment
0,2024-06-08 21:47:00.468,torch,pythia1-4b,0,2,24564,65,11.65,500.0,37,...,0.264615,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397,193.242,34523.835300,1
1,2024-06-08 21:47:00.571,torch,pythia1-4b,0,2,24564,65,11.66,500.0,37,...,0.264615,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397,193.242,34523.835300,1
2,2024-06-08 21:47:00.672,torch,pythia1-4b,0,2,24564,65,11.67,500.0,37,...,0.264615,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397,193.242,34523.835300,1
3,2024-06-08 21:47:00.773,torch,pythia1-4b,0,2,24564,65,11.68,500.0,37,...,0.264615,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397,193.242,34523.835300,1
4,2024-06-08 21:47:00.874,torch,pythia1-4b,0,2,24564,65,11.70,500.0,37,...,0.264615,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397,193.242,34523.835300,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354446,2024-06-11 19:27:33.098,torchscript,phi2,13,8,24564,14604,75.50,500.0,43,...,59.452858,5.393911,2.355166,14531.867159,73.543976,44.414207,59.159205,109.296,8038.062403,9
354447,2024-06-11 19:27:33.199,torchscript,phi2,0,0,24564,14604,75.53,500.0,42,...,59.452858,5.393911,2.355166,14531.867159,73.543976,44.414207,59.159205,109.296,8038.062403,9
354448,2024-06-11 19:27:33.307,torchscript,phi2,0,0,24564,14604,75.61,500.0,42,...,59.452858,5.393911,2.355166,14531.867159,73.543976,44.414207,59.159205,109.296,8038.062403,9
354449,2024-06-11 19:27:33.408,torchscript,phi2,0,0,24564,14604,75.67,500.0,42,...,59.452858,5.393911,2.355166,14531.867159,73.543976,44.414207,59.159205,109.296,8038.062403,9


In [24]:
condition = nvidia_df['label'].str.endswith('phi2')
nvidia_df = remove_condition_rows( condition, nvidia_df)
nvidia_df


,time,runtime,model,utilization.gpu [%],utilization.memory [%],memory.total [MiB],memory.used [MiB],power.draw [W],power.max_limit [W],temperature.gpu,...,used_memory_pct_mib,avg_utilization_gpu_config,avg_utilization_memory_config,avg_memory_used_config,avg_power_draw_config,avg_temperature_gpu_config,avg_used_memory_pct_mib,config_duration,energy,experiment
0,2024-06-08 21:47:00.468,torch,pythia1-4b,0,2,24564,65,11.65,500.0,37,...,0.264615,73.584680,67.165711,6061.01876,178.655961,49.400730,24.674397,193.242,34523.835300,1
1,2024-06-08 21:47:00.571,torch,pythia1-4b,0,2,24564,65,11.66,500.0,37,...,0.264615,73.584680,67.165711,6061.01876,178.655961,49.400730,24.674397,193.242,34523.835300,1
2,2024-06-08 21:47:00.672,torch,pythia1-4b,0,2,24564,65,11.67,500.0,37,...,0.264615,73.584680,67.165711,6061.01876,178.655961,49.400730,24.674397,193.242,34523.835300,1
3,2024-06-08 21:47:00.773,torch,pythia1-4b,0,2,24564,65,11.68,500.0,37,...,0.264615,73.584680,67.165711,6061.01876,178.655961,49.400730,24.674397,193.242,34523.835300,1
4,2024-06-08 21:47:00.874,torch,pythia1-4b,0,2,24564,65,11.70,500.0,37,...,0.264615,73.584680,67.165711,6061.01876,178.655961,49.400730,24.674397,193.242,34523.835300,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353362,2024-06-11 19:25:43.548,torchscript,pythia-410m,3,1,24564,13602,62.51,500.0,47,...,55.373718,1.213198,0.327919,13602.55736,64.916294,52.826396,55.375987,99.293,6445.733621,9
353363,2024-06-11 19:25:43.649,torchscript,pythia-410m,0,0,24564,13602,62.50,500.0,47,...,55.373718,1.213198,0.327919,13602.55736,64.916294,52.826396,55.375987,99.293,6445.733621,9
353364,2024-06-11 19:25:43.750,torchscript,pythia-410m,0,0,24564,13602,62.46,500.0,47,...,55.373718,1.213198,0.327919,13602.55736,64.916294,52.826396,55.375987,99.293,6445.733621,9
353365,2024-06-11 19:25:43.851,torchscript,pythia-410m,0,0,24564,13602,62.40,500.0,47,...,55.373718,1.213198,0.327919,13602.55736,64.916294,52.826396,55.375987,99.293,6445.733621,9


In [25]:
nvidia_df.columns

Index(['time', 'runtime', 'model', ' utilization.gpu [%]',
       ' utilization.memory [%]', ' memory.total [MiB]', ' memory.used [MiB]',
       ' power.draw [W]', ' power.max_limit [W]', ' temperature.gpu', 'label',
       'used_memory_pct_mib', 'avg_utilization_gpu_config',
       'avg_utilization_memory_config', 'avg_memory_used_config',
       'avg_power_draw_config', 'avg_temperature_gpu_config',
       'avg_used_memory_pct_mib', 'config_duration', 'energy', 'experiment'],
      dtype='object')

In [26]:
gpu_energy_aggregated = nvidia_df.groupby(['label', 'experiment'])['energy'].mean().reset_index()
gpu_energy_aggregated

,label,experiment,energy
0,onnx_codeparrot-small,1,15938.552931
1,onnx_codeparrot-small,2,13823.850824
2,onnx_codeparrot-small,3,13896.811929
3,onnx_codeparrot-small,4,16083.457295
4,onnx_codeparrot-small,5,13918.800235
...,...,...,...
115,torchscript_tinyllama,6,7022.551625
116,torchscript_tinyllama,7,7052.688061
117,torchscript_tinyllama,8,6716.034659
118,torchscript_tinyllama,9,7063.159418


In [27]:
# Initialize an empty DataFrame to collect test results
results_df = pd.DataFrame(columns=['config', 'W-statistic', 'p-value'])

# Iterate over each configuration
for c in configs:
    # Filter data for current configuration
    subset = gpu_energy_aggregated[gpu_energy_aggregated['label'].str.startswith(c, na=False)]
    
    # Perform the Shapiro-Wilk test
    w_statistic, p_value = shapiro(subset['energy'])
    
    # Prepare a dictionary to add as a new row
    new_row = pd.DataFrame({'config': [c], 'W-statistic': [w_statistic], 'p-value': [p_value]})
    
    # Concatenate the new row to the results DataFrame
    results_df = pd.concat([results_df, new_row], ignore_index=True)

# Display the results DataFrame
print(results_df)



                          config  W-statistic   p-value
0               torch_pythia1-4b     0.881052  0.134171
1                torch_tinyllama     0.635806  0.000148
2         torch_codeparrot-small     0.803865  0.016154
3              torch_pythia-410m     0.702893  0.000955
4                onnx_pythia1-4b     0.705650  0.001031
5                 onnx_tinyllama     0.753470  0.003933
6          onnx_codeparrot-small     0.786519  0.009939
7               onnx_pythia-410m     0.874583  0.113008
8         torchscript_pythia1-4b     0.698376  0.000842
9          torchscript_tinyllama     0.752027  0.003777
10  torchscript_codeparrot-small     0.840937  0.045280
11       torchscript_pythia-410m     0.868253  0.095382


In [28]:
save_latex_table(results_df,save_dir + "gpu_energy_shapiro")


\begin{tabular}{lrr}
\toprule
config & W-statistic & p-value \\
\midrule
torch_pythia1-4b & 0.881052 & 0.134171 \\
torch_tinyllama & 0.635806 & 0.000148 \\
torch_codeparrot-small & 0.803865 & 0.016154 \\
torch_pythia-410m & 0.702893 & 0.000955 \\
onnx_pythia1-4b & 0.705650 & 0.001031 \\
onnx_tinyllama & 0.753470 & 0.003933 \\
onnx_codeparrot-small & 0.786519 & 0.009939 \\
onnx_pythia-410m & 0.874583 & 0.113008 \\
torchscript_pythia1-4b & 0.698376 & 0.000842 \\
torchscript_tinyllama & 0.752027 & 0.003777 \\
torchscript_codeparrot-small & 0.840937 & 0.045280 \\
torchscript_pythia-410m & 0.868253 & 0.095382 \\
\bottomrule
\end{tabular}



In [29]:
variable_aggregated = gpu_energy_aggregated
sets_levene = []
for c in configs:
    sets_levene.append(variable_aggregated[variable_aggregated['label'].str.startswith(c, na=False)]['energy'])

In [30]:
results_df = pd.DataFrame(columns=['W-statistic', 'p-value'])
w_statistic, p_value = levene(*sets_levene)
    
# Prepare a dictionary to add as a new row
new_row = pd.DataFrame({'W-statistic': [w_statistic], 'p-value': [p_value]})

# Concatenate the new row to the results DataFrame
results_df = pd.concat([results_df, new_row], ignore_index=True)

save_latex_table(results_df,save_dir + 'gpu_energy_levene')
# Display the results DataFrame
print(results_df)

\begin{tabular}{rr}
\toprule
W-statistic & p-value \\
\midrule
3.933012 & 0.000083 \\
\bottomrule
\end{tabular}

   W-statistic   p-value
0     3.933012  0.000083


## global energy

In [ ]:
path = "D:/GAISSA/energy-repo/last_repo/june_gpu/"  # Adjust this path to your CSV files location
all_files_nvidia = glob.glob(path + "/results_*/processed/nvidia*_processed.csv")
all_files_nvidia

In [ ]:
df = pd.read_csv(my_file, index_col=None, header=0)

In [31]:
#global_energy = merged_df_plot.to_csv(tables_dir+'final_energy_data.csv', index=False)
global_energy = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_gpu/tables/final_energy_data.csv", index_col=None, header=0)
global_energy

,label,experiment,CPU_energy,GPU_energy,global_energy
0,idle_idle,1,5356.125122,0.000000,5356.125122
1,idle_idle,2,5350.155334,0.000000,5350.155334
2,idle_idle,3,5174.852402,0.000000,5174.852402
3,idle_idle,4,5180.039169,0.000000,5180.039169
4,idle_idle,5,5366.871078,0.000000,5366.871078
...,...,...,...,...,...
125,torchscript_tinyllama,6,2479.216080,7022.551625,9501.767705
126,torchscript_tinyllama,7,2528.263107,7052.688061,9580.951168
127,torchscript_tinyllama,8,2645.645905,6716.034659,9361.680564
128,torchscript_tinyllama,9,2569.570862,7063.159418,9632.730280


In [32]:
# Initialize an empty DataFrame to collect test results
results_df = pd.DataFrame(columns=['config', 'W-statistic', 'p-value'])

# Iterate over each configuration
for c in configs:
    # Filter data for current configuration
    subset = global_energy[global_energy['label'].str.startswith(c, na=False)]
    
    # Perform the Shapiro-Wilk test
    w_statistic, p_value = shapiro(subset['global_energy'])
    
    # Prepare a dictionary to add as a new row
    new_row = pd.DataFrame({'config': [c], 'W-statistic': [w_statistic], 'p-value': [p_value]})
    
    # Concatenate the new row to the results DataFrame
    results_df = pd.concat([results_df, new_row], ignore_index=True)

# Display the results DataFrame
print(results_df)



                          config  W-statistic   p-value
0               torch_pythia1-4b     0.875566  0.116009
1                torch_tinyllama     0.638411  0.000159
2         torch_codeparrot-small     0.878973  0.126996
3              torch_pythia-410m     0.691307  0.000691
4                onnx_pythia1-4b     0.738626  0.002594
5                 onnx_tinyllama     0.808198  0.018234
6          onnx_codeparrot-small     0.851734  0.060914
7               onnx_pythia-410m     0.879445  0.128592
8         torchscript_pythia1-4b     0.540852  0.000011
9          torchscript_tinyllama     0.887527  0.159015
10  torchscript_codeparrot-small     0.952964  0.703622
11       torchscript_pythia-410m     0.888609  0.163557


In [34]:
variable_aggregated = global_energy
sets_levene = []
for c in configs:
    sets_levene.append(variable_aggregated[variable_aggregated['label'].str.startswith(c, na=False)]['global_energy'])

In [35]:
results_df = pd.DataFrame(columns=['W-statistic', 'p-value'])
w_statistic, p_value = levene(*sets_levene)
    
# Prepare a dictionary to add as a new row
new_row = pd.DataFrame({'W-statistic': [w_statistic], 'p-value': [p_value]})

# Concatenate the new row to the results DataFrame
results_df = pd.concat([results_df, new_row], ignore_index=True)

save_latex_table(results_df,save_dir + 'global_energy_levene')
# Display the results DataFrame
print(results_df)

\begin{tabular}{rr}
\toprule
W-statistic & p-value \\
\midrule
4.275798 & 0.000028 \\
\bottomrule
\end{tabular}

   W-statistic   p-value
0     4.275798  0.000028


In [36]:
from scipy import stats

In [41]:
import scipy.stats as stats

def kruskal_wallis_test(*groups, alpha=0.05):
    # Perform the Kruskal-Wallis test
    stat, p_value = stats.kruskal(*groups)
    
    # Print the test statistic and p-value
    print(f"Kruskal-Wallis H statistic: {stat:}")
    print(f"P-value: {p_value}")
    
    # Interpretation based on the p-value
    if p_value < alpha:
        print(f"Since the p-value ({p_value}) is less than {alpha}, we reject the null hypothesis.")
        print("There is statistically significant difference between the groups.")
    else:
        print(f"Since the p-value ({p_value}) is greater than {alpha}, we fail to reject the null hypothesis.")
        print("There is no statistically significant difference between the groups.")


In [42]:
kruskal_wallis_test(*sets_levene)

Kruskal-Wallis H statistic: 117.88264462809929
P-value: 4.8305727765486224e-20
Since the p-value (4.8305727765486224e-20) is less than 0.05, we reject the null hypothesis.
There is statistically significant difference between the groups.


In [55]:
import scipy.stats as stats
import scikit_posthocs as sp
import pandas as pd
import numpy as np

In [58]:
def kruskal_wallis_with_dunn(*groups, alpha=0.05):
    # Perform the Kruskal-Wallis test
    stat, p_value = stats.kruskal(*groups)
    print(f"Kruskal-Wallis H statistic: {stat:}")
    print(f"P-value: {p_value:}")
    
    # Interpret the Kruskal-Wallis result
    if p_value < alpha:
        print(f"Since the p-value ({p_value:}) is less than {alpha}, we reject the null hypothesis.")
        print("There is statistically significant difference between the groups.")
        print("Proceeding with Dunn's post-hoc test...")
        
        # Flatten the group lists and create a grouping array for Dunn's test
        data = np.concatenate(groups)
        group_arr = np.concatenate([[i] * len(group) for i, group in enumerate(groups, 1)])
        
        # Perform Dunn's test using scikit-posthocs
        #dunn_df = sp.posthoc_dunn(group_arr, data, p_adjust='bonferroni')
        dunn_df = sp.posthoc_dunn(group_arr, data, p_adjust="holm")
        
        print("Dunn's post-hoc test pairwise p-values:")
        print(dunn_df)
    else:
        print(f"Since the p-value ({p_value:}) is greater than {alpha}, we fail to reject the null hypothesis.")
        print("There is no statistically significant difference between the groups.")


In [64]:
def kruskal_wallis_with_dunn(*groups, alpha=0.05):
    # Flatten the group lists and create a grouping array
    data = np.concatenate(groups)
    group_arr = np.concatenate([[i + 1] * len(group) for i, group in enumerate(groups)])  # +1 to ensure group labels are not zero
    
    # Perform the Kruskal-Wallis test
    stat, p_value = stats.kruskal(*groups)
    print(f"Kruskal-Wallis H statistic: {stat:.3f}")
    print(f"P-value: {p_value:.3f}")
    
    if p_value < alpha:
        print("Reject the null hypothesis: significant differences between the groups.")
        print("Proceeding with Dunn's post-hoc test...")
        
        # Prepare DataFrame for Dunn's test
        df = pd.DataFrame({'values': data, 'groups': group_arr})
        
        # Perform Dunn's test using scikit-posthocs
        dunn_results = sp.posthoc_dunn(df, val_col='values', group_col='groups', p_adjust='bonferroni')
        
        print("Dunn's post-hoc test pairwise p-values:")
        print(dunn_results)
        print(dunn_results < 0.05)
        
    else:
        print("Fail to reject the null hypothesis: no significant differences between the groups.")


In [65]:
kruskal_wallis_with_dunn(*sets_levene)

Kruskal-Wallis H statistic: 117.883
P-value: 0.000
Reject the null hypothesis: significant differences between the groups.
Proceeding with Dunn's post-hoc test...
Dunn's post-hoc test pairwise p-values:
          1         2         3         4             5             6   \
1   1.000000  1.000000  0.720027  1.000000  1.000000e+00  1.000000e+00   
2   1.000000  1.000000  1.000000  1.000000  7.200269e-01  1.000000e+00   
3   0.720027  1.000000  1.000000  1.000000  4.489775e-04  7.577623e-03   
4   1.000000  1.000000  1.000000  1.000000  8.444362e-02  6.563893e-01   
5   1.000000  0.720027  0.000449  0.084444  1.000000e+00  1.000000e+00   
6   1.000000  1.000000  0.007578  0.656389  1.000000e+00  1.000000e+00   
7   1.000000  1.000000  1.000000  1.000000  7.779311e-03  8.831065e-02   
8   1.000000  1.000000  0.086357  1.000000  1.000000e+00  1.000000e+00   
9   0.011780  0.105469  1.000000  0.863963  7.821841e-07  2.781239e-05   
10  0.070487  0.485069  1.000000  1.000000  1.142228e-05 

# Wattmeter metrics

In [41]:
# Path where the CSV files are stored
path = "D:/GAISSA/energy-repo/last_repo/june_gpu/"  # Adjust this path to your CSV files location
all_files_wattmeter = glob.glob(path + "/results_*/processed/wattmeter*_processed.csv")
all_files_wattmeter

['D:/GAISSA/energy-repo/last_repo/june_gpu\\results_1\\processed\\wattmeter_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_10\\processed\\wattmeter_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_2\\processed\\wattmeter_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_3\\processed\\wattmeter_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_4\\processed\\wattmeter_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_5\\processed\\wattmeter_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_6\\processed\\wattmeter_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_7\\processed\\wattmeter_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_8\\processed\\wattmeter_processed.csv',
 'D:/GAISSA/energy-repo/last_repo/june_gpu\\results_9\\processed\\wattmeter_processed.csv']

In [42]:
result_numbers = []

li = []
for my_file in all_files_wattmeter:
    # The regex pattern to find "results_" followed by one or more digits
    df = pd.read_csv(my_file, index_col=None, header=0)
    
    match = re.search(r'results_(\d+)', my_file)
    if match:
        # Append the found number to the list
        result_numbers.append(int(match.group(1)))

    df['experiment'] = result_numbers[-1]
    li.append(df)

print(result_numbers)
# Combine all data into one DataFrame
wattmeter_df = pd.concat(li, axis=0, ignore_index=True)
wattmeter_df

[1, 10, 2, 3, 4, 5, 6, 7, 8, 9]


,time,runtime,model,Current,PowerFactor,Phase,Energy,EnergyNR,Load,label,energy_joules,energyNR_joules,energy_joules_config,energyNR_joules_config,avg_Current,avg_PowerFactor,avg_Phase,avg_Load,experiment
0,2024-06-08 21:47:02.854518,torch,pythia1-4b,717,1.00,336.26,120086,120086,174,torch_pythia1-4b,432309600,432309600,54000,54000,1278.477341,0.931662,343.706465,284.371601,1
1,2024-06-08 21:47:03.429127,torch,pythia1-4b,717,1.00,336.26,120086,120086,174,torch_pythia1-4b,432309600,432309600,54000,54000,1278.477341,0.931662,343.706465,284.371601,1
2,2024-06-08 21:47:04.017249,torch,pythia1-4b,812,0.91,335.38,120086,120086,176,torch_pythia1-4b,432309600,432309600,54000,54000,1278.477341,0.931662,343.706465,284.371601,1
3,2024-06-08 21:47:04.591239,torch,pythia1-4b,812,0.91,335.38,120086,120086,176,torch_pythia1-4b,432309600,432309600,54000,54000,1278.477341,0.931662,343.706465,284.371601,1
4,2024-06-08 21:47:05.163254,torch,pythia1-4b,812,0.91,335.38,120086,120086,176,torch_pythia1-4b,432309600,432309600,54000,54000,1278.477341,0.931662,343.706465,284.371601,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41433,2024-06-11 19:27:30.645104,torchscript,phi2,863,0.64,325.91,128705,128705,132,torchscript_phi2,463338000,463338000,18000,18000,757.942529,0.779713,323.222471,150.931034,9
41434,2024-06-11 19:27:31.221117,torchscript,phi2,863,0.64,325.91,128705,128705,132,torchscript_phi2,463338000,463338000,18000,18000,757.942529,0.779713,323.222471,150.931034,9
41435,2024-06-11 19:27:31.799037,torchscript,phi2,695,0.73,326.06,128705,128705,123,torchscript_phi2,463338000,463338000,18000,18000,757.942529,0.779713,323.222471,150.931034,9
41436,2024-06-11 19:27:32.378849,torchscript,phi2,695,0.73,326.06,128705,128705,123,torchscript_phi2,463338000,463338000,18000,18000,757.942529,0.779713,323.222471,150.931034,9


In [43]:
condition = wattmeter_df['label'].str.endswith('phi2')

wattmeter_df = remove_condition_rows( condition, wattmeter_df)
wattmeter_df

,time,runtime,model,Current,PowerFactor,Phase,Energy,EnergyNR,Load,label,energy_joules,energyNR_joules,energy_joules_config,energyNR_joules_config,avg_Current,avg_PowerFactor,avg_Phase,avg_Load,experiment
0,2024-06-08 21:47:02.854518,torch,pythia1-4b,717,1.00,336.26,120086,120086,174,torch_pythia1-4b,432309600,432309600,54000,54000,1278.477341,0.931662,343.706465,284.371601,1
1,2024-06-08 21:47:03.429127,torch,pythia1-4b,717,1.00,336.26,120086,120086,174,torch_pythia1-4b,432309600,432309600,54000,54000,1278.477341,0.931662,343.706465,284.371601,1
2,2024-06-08 21:47:04.017249,torch,pythia1-4b,812,0.91,335.38,120086,120086,176,torch_pythia1-4b,432309600,432309600,54000,54000,1278.477341,0.931662,343.706465,284.371601,1
3,2024-06-08 21:47:04.591239,torch,pythia1-4b,812,0.91,335.38,120086,120086,176,torch_pythia1-4b,432309600,432309600,54000,54000,1278.477341,0.931662,343.706465,284.371601,1
4,2024-06-08 21:47:05.163254,torch,pythia1-4b,812,0.91,335.38,120086,120086,176,torch_pythia1-4b,432309600,432309600,54000,54000,1278.477341,0.931662,343.706465,284.371601,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41259,2024-06-11 19:25:41.122919,torchscript,pythia-410m,649,0.81,325.63,128700,128700,127,torchscript_pythia-410m,463320000,463320000,14400,14400,680.798817,0.837515,325.907041,138.136095,9
41260,2024-06-11 19:25:41.702703,torchscript,pythia-410m,649,0.81,325.63,128700,128700,127,torchscript_pythia-410m,463320000,463320000,14400,14400,680.798817,0.837515,325.907041,138.136095,9
41261,2024-06-11 19:25:42.289644,torchscript,pythia-410m,643,0.83,324.62,128700,128700,128,torchscript_pythia-410m,463320000,463320000,14400,14400,680.798817,0.837515,325.907041,138.136095,9
41262,2024-06-11 19:25:42.865670,torchscript,pythia-410m,643,0.83,324.62,128700,128700,128,torchscript_pythia-410m,463320000,463320000,14400,14400,680.798817,0.837515,325.907041,138.136095,9


In [52]:
wattmeter_df.columns

Index(['time', 'runtime', 'model', 'Current', 'PowerFactor', 'Phase', 'Energy',
       'EnergyNR', 'Load', 'label', 'energy_joules', 'energyNR_joules',
       'energy_joules_config', 'energyNR_joules_config', 'avg_Current',
       'avg_PowerFactor', 'avg_Phase', 'avg_Load', 'experiment'],
      dtype='object')

In [54]:
variable = 'energy_joules_config'
variable_aggregated = wattmeter_df.groupby(['label', 'experiment'])[variable].mean().reset_index()
variable_aggregated

,label,experiment,energy_joules_config
0,onnx_codeparrot-small,1,32400.0
1,onnx_codeparrot-small,2,32400.0
2,onnx_codeparrot-small,3,32400.0
3,onnx_codeparrot-small,4,36000.0
4,onnx_codeparrot-small,5,36000.0
...,...,...,...
115,torchscript_tinyllama,6,18000.0
116,torchscript_tinyllama,7,14400.0
117,torchscript_tinyllama,8,14400.0
118,torchscript_tinyllama,9,14400.0


In [56]:
# Initialize an empty DataFrame to collect test results
results_df = pd.DataFrame(columns=['config', 'W-statistic', 'p-value'])

# Iterate over each configuration
for c in configs:
    # Filter data for current configuration
    subset = variable_aggregated[variable_aggregated['label'].str.startswith(c, na=False)]
    
    # Perform the Shapiro-Wilk test
    w_statistic, p_value = shapiro(subset[variable])
    
    # Prepare a dictionary to add as a new row
    new_row = pd.DataFrame({'config': [c], 'W-statistic': [w_statistic], 'p-value': [p_value]})
    
    # Concatenate the new row to the results DataFrame
    results_df = pd.concat([results_df, new_row], ignore_index=True)

# Display the results DataFrame
print(variable)
print(results_df)



energy_joules_config
                          config  W-statistic       p-value
0               torch_pythia1-4b     0.594174  4.713490e-05
1                torch_tinyllama     0.794064  1.227873e-02
2         torch_codeparrot-small     0.365721  1.003693e-07
3              torch_pythia-410m     0.594174  4.713490e-05
4                onnx_pythia1-4b     0.770498  6.341842e-03
5                 onnx_tinyllama     0.872879  1.079800e-01
6          onnx_codeparrot-small     0.640485  1.686800e-04
7               onnx_pythia-410m     0.806909  1.758880e-02
8         torchscript_pythia1-4b     0.700495  8.931572e-04
9          torchscript_tinyllama     0.509275  4.672362e-06
10  torchscript_codeparrot-small     0.594174  4.713490e-05
11       torchscript_pythia-410m     0.594174  4.713490e-05


In [57]:
save_latex_table(results_df,save_dir + f"wattmeter_{variable}_shapiro")

\begin{tabular}{lrr}
\toprule
config & W-statistic & p-value \\
\midrule
torch_pythia1-4b & 0.594174 & 0.000047 \\
torch_tinyllama & 0.794064 & 0.012279 \\
torch_codeparrot-small & 0.365721 & 0.000000 \\
torch_pythia-410m & 0.594174 & 0.000047 \\
onnx_pythia1-4b & 0.770498 & 0.006342 \\
onnx_tinyllama & 0.872879 & 0.107980 \\
onnx_codeparrot-small & 0.640485 & 0.000169 \\
onnx_pythia-410m & 0.806909 & 0.017589 \\
torchscript_pythia1-4b & 0.700495 & 0.000893 \\
torchscript_tinyllama & 0.509275 & 0.000005 \\
torchscript_codeparrot-small & 0.594174 & 0.000047 \\
torchscript_pythia-410m & 0.594174 & 0.000047 \\
\bottomrule
\end{tabular}



In [58]:
variable_aggregated = variable_aggregated
sets_levene = []
for c in configs:
    sets_levene.append(variable_aggregated[variable_aggregated['label'].str.startswith(c, na=False)][variable])

In [59]:
results_df = pd.DataFrame(columns=['W-statistic', 'p-value'])
w_statistic, p_value = levene(*sets_levene)
    
# Prepare a dictionary to add as a new row
new_row = pd.DataFrame({'W-statistic': [w_statistic], 'p-value': [p_value]})

# Concatenate the new row to the results DataFrame
results_df = pd.concat([results_df, new_row], ignore_index=True)

save_latex_table(results_df,save_dir + f'wattmeter_{variable}_levene')
# Display the results DataFrame
print(results_df)

\begin{tabular}{rr}
\toprule
W-statistic & p-value \\
\midrule
1.604959 & 0.107346 \\
\bottomrule
\end{tabular}

   W-statistic   p-value
0     1.604959  0.107346
